In [1]:
!pip install ydf -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 28.5 MB/s eta 0:00:00


In [2]:
import ydf
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
data_train, data_test= train_test_split(pd.read_csv("/kaggle/input/titanic/train.csv"))
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
793,794,0,1,"Hoyt, Mr. William Fisher",male,NaN,0,0,PC 17600,30.6958,NaN,C
526,527,1,2,"Ridsdale, Miss. Lucy",female,50.0,0,0,W./C. 14258,10.5000,NaN,S
16,17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.1250,NaN,Q
36,37,1,3,"Mamee, Mr. Hanna",male,NaN,0,0,2677,7.2292,NaN,C
260,261,0,3,"Smith, Mr. Thomas",male,NaN,0,0,384461,7.7500,NaN,Q


In [4]:
data_train = data_train.drop(columns=['Name','PassengerId'])
data_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
793,0,1,male,NaN,0,0,PC 17600,30.6958,NaN,C
526,1,2,female,50.0,0,0,W./C. 14258,10.5000,NaN,S
16,0,3,male,2.0,4,1,382652,29.1250,NaN,Q
36,1,3,male,NaN,0,0,2677,7.2292,NaN,C
260,0,3,male,NaN,0,0,384461,7.7500,NaN,Q


In [5]:
model = ydf.GradientBoostedTreesLearner(label="Survived").train(data_train)

Train model on 668 examples


max_vocab_count = -1 for column Survived, the dictionary will not be pruned by size.
"goss_alpha" set but "sampling_method" not equal to "GOSS".
"goss_beta" set but "sampling_method" not equal to "GOSS".
"selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
Data spec:
Number of records: 668
Number of columns: 10

Number of columns by type:
	CATEGORICAL: 5 (50%)
	NUMERICAL: 5 (50%)

Columns:

CATEGORICAL: 5 (50%)
	0: "Survived" CATEGORICAL has-dict vocab-size:3 zero-ood-items most-frequent:"0" 414 (61.976%) dtype:DTYPE_INT64
	2: "Sex" CATEGORICAL has-dict vocab-size:3 zero-ood-items most-frequent:"male" 436 (65.2695%) dtype:DTYPE_BYTES
	6: "Ticket" CATEGORICAL has-dict vocab-size:7 num-oods:632 (94.6108%) most-frequent:"<OOD>" 632 (94.6108%) dtype:DTYPE_BYTES
	8: "Cabin" CATEGORICAL num-nas:504 (75.4491%) has-dict vocab-size:1 num-oods:164 (100%) dtype:DTYPE_BYTES
	9: "Embarked" CATEGORICAL num-nas:1 (0.149701%) has-dict vocab-size:4 zero-ood-items most-fre

Model trained in 0:00:00.312036


Early stop of the training because the validation loss does not decrease anymore. Best valid-loss: 0.671515
Truncates the model to 88 tree(s) i.e. 88  iteration(s).
Final model num-trees:88 valid-loss:0.671515 valid-accuracy:0.887097


In [6]:
# Look at a model (input features, training logs, structure, etc.)
model.describe()

In [7]:
data_test = data_test.drop(columns=['Name','PassengerId'])
data_test.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
229,0,3,female,NaN,3,1,4133,25.4667,NaN,S
797,1,3,female,31.0,0,0,349244,8.6833,NaN,S
91,0,3,male,20.0,0,0,347466,7.8542,NaN,S
290,1,1,female,26.0,0,0,19877,78.8500,NaN,S
20,0,2,male,35.0,0,0,239865,26.0000,NaN,S


In [8]:
model.evaluate(data_test)

Engine "GradientBoostedTreesQuickScorerExtended" built


Label \ Pred,0,1
0,124,26
1,11,62


In [9]:
model.analyze(data_test)

Run the model with the fast engine wrapper
List plotting attribute combinations
Found 9 combination(s)
Initiate PDP accumulator
Compute partial dependence plot for 9 set of features and 105 model call(s) per example.
1 examples scanned.
List plotting attribute combinations
Found 9 combination(s)
Initiate CEP accumulator
Compute conditional expectation plot for 9 set of features and 105 model call(s) per example.
1 examples scanned.
Running 10 features on 6 threads with 1 rounds
Model self evaluation:
Task: CLASSIFICATION
Label: Survived
Loss (BINOMIAL_LOG_LIKELIHOOD): 0.671515

Accuracy: 0.887097  CI95[W][0 1]
ErrorRate: : 0.112903


Confusion Table:
truth\prediction
        0   1
    0  36   2
    1   5  19
Total: 62




In [10]:
submission = pd.read_csv("/kaggle/input/titanic/test.csv")
submission.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [11]:
y_pred = [1 if s > 0.5 else 0 for s in model.predict(data_test) ]
accuracy = accuracy_score(data_test['Survived'], y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.83


In [12]:
predictions = model.predict(submission)
submission['Survived'] = [1 if s > 0.5 else 0 for s in predictions]
submission = submission[['PassengerId', 'Survived']]

In [13]:
submission.to_csv('submission.csv', index=False)
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
